In [96]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from mlxtend.preprocessing import TransactionEncoder
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [97]:
# load data
data = pd.read_csv('../data/basket/Assignment-1_Data.csv', delimiter=';')
data.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [98]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  object 
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [99]:
# 数据清洗
data.dropna(subset=['Itemname'], inplace=True)
data['Price'] = data['Price'].str.replace(',','.').astype('float64')
# sns.boxplot(data[['price']])
data = data[data['Price'] > 0]
data = data[data['Country'] == 'United Kingdom']
data['Total price']=data.Quantity * data.Price
data.info()
#data = data.dropna()
#data = data.drop_duplicates()

<class 'pandas.core.frame.DataFrame'>
Index: 485123 entries, 0 to 522048
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   BillNo       485123 non-null  object 
 1   Itemname     485123 non-null  object 
 2   Quantity     485123 non-null  int64  
 3   Date         485123 non-null  object 
 4   Price        485123 non-null  float64
 5   CustomerID   354321 non-null  float64
 6   Country      485123 non-null  object 
 7   Total price  485123 non-null  float64
dtypes: float64(3), int64(1), object(4)
memory usage: 33.3+ MB


In [100]:
data = data.loc[
    (data['Itemname']!= 'Adjust bad debt') & 
    (data['Itemname']!= 'AMAZON FEE') & 
    (data['Itemname']!= 'Bank Charges') &
    (data['Itemname']!= 'DOTCOM POSTAGE') & 
    (data['Itemname']!= 'Dotcomgiftshop Gift Voucher Â£10.00') &
    (data['Itemname']!= 'Dotcomgiftshop Gift Voucher Â£100.00') &
    (data['Itemname']!= 'Dotcomgiftshop Gift Voucher Â£20.00') & 
    (data['Itemname']!= 'Dotcomgiftshop Gift Voucher Â£30.00') & 
    (data['Itemname']!= 'Dotcomgiftshop Gift Voucher Â£40.00') & 
    (data['Itemname']!= 'Dotcomgiftshop Gift Voucher Â£50.00') & 
    (data['Itemname']!= 'Manual') & 
    (data['Itemname']!= 'POSTAGE') &
    (data['Itemname']!= 'SAMPLES')
]

In [101]:
data['Country'].value_counts()

Country
United Kingdom    484036
Name: count, dtype: int64

In [102]:
data['Year']=data['Date'].apply(lambda x:x.split('.')[2])
data['Year']=data['Year'].apply(lambda x:x.split(' ')[0])
data['Month']=data['Date'].apply(lambda x:x.split('.')[1])

data.groupby(['Year','Month'])['Total price'].sum()

Year  Month
2010  12        707469.610
2011  01        547167.610
      02        418983.950
      03        573123.130
      04        466182.151
      05        619541.150
      06        606270.960
      07        578274.681
      08        579975.270
      09        878329.111
      10        914878.250
      11       1286474.780
      12        572282.680
Name: Total price, dtype: float64

In [103]:
data=data.loc[data['Year']!='2010']

In [104]:
# 使用value_counts()对Category列进行分类统计
#category_counts = data['Itemname'].value_counts().reset_index()
#category_counts.columns = ['Itemname', 'Count']

# 按照分类统计结果创建新的DataFrame
#category_counts_df = pd.DataFrame(category_counts)

# 将分类统计结果保存到本地文件，比如CSV格式
#category_counts_df.to_csv('category_counts.csv', index=False)

In [105]:
data.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country,Total price,Year,Month
41242,539993,JUMBO BAG PINK POLKADOT,10,04.01.2011 10:00,1.95,13313.0,United Kingdom,19.5,2011,01
41243,539993,BLUE POLKADOT WRAP,25,04.01.2011 10:00,0.42,13313.0,United Kingdom,10.5,2011,01
41244,539993,RED RETROSPOT WRAP,25,04.01.2011 10:00,0.42,13313.0,United Kingdom,10.5,2011,01
41245,539993,RECYCLING BAG RETROSPOT,5,04.01.2011 10:00,2.10,13313.0,United Kingdom,10.5,2011,01
41246,539993,RED RETROSPOT SHOPPER BAG,10,04.01.2011 10:00,1.25,13313.0,United Kingdom,12.5,2011,01


In [106]:
data = data[data['Country']=='United Kingdom']
transactions = data.groupby('BillNo')['Itemname'].apply(list).tolist()
transactions = [transaction for transaction in transactions if len(transaction) >= 2]
len(transactions)

15228

In [107]:
# One-hot encode transactions
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df_encoded = pd.DataFrame(te_ary, columns=te.columns_)

In [108]:
# Apply Apriori algorithm
frequent_itemsets = apriori(df_encoded, min_support=0.02, use_colnames=True, low_memory=True)

In [109]:
# Print frequent itemsets
print(frequent_itemsets)

      support                                           itemsets
0    0.022065                 (3 HOOK PHOTO SHELF ANTIQUE WHITE)
1    0.025873                         (3 STRIPEY MICE FELTCRAFT)
2    0.023115                      (4 TRADITIONAL SPINNING TOPS)
3    0.051681                           (6 RIBBONS RUSTIC CHARM)
4    0.022459                  (60 CAKE CASES DOLLY GIRL DESIGN)
..        ...                                                ...
469  0.023378  (JUMBO SHOPPER VINTAGE RED PAISLEY, JUMBO BAG ...
470  0.025479  (JUMBO BAG RED RETROSPOT, JUMBO STORAGE BAG SU...
471  0.024297  (JUMBO BAG RED RETROSPOT, JUMBO SHOPPER VINTAG...
472  0.021671  (LUNCH BAG PINK POLKADOT, LUNCH BAG RED RETROS...
473  0.020489  (LUNCH BAG SUKI DESIGN, LUNCH BAG RED RETROSPO...

[474 rows x 2 columns]


In [111]:
# 使用关联规则查找频繁项集
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.7)
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(CHARLOTTE BAG PINK POLKADOT),(RED RETROSPOT CHARLOTTE BAG),0.041240,0.055687,0.029157,0.707006,12.696100,0.026860,3.222982,0.960861
1,(GARDENERS KNEELING PAD CUP OF TEA),(GARDENERS KNEELING PAD KEEP CALM),0.049186,0.058905,0.035527,0.722296,12.262129,0.032629,3.388848,0.965959
2,(PINK REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.045640,0.057197,0.037628,0.824460,14.414332,0.035018,5.370884,0.975129
3,(GREEN REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.057197,0.058314,0.042882,0.749713,12.856564,0.039546,3.762426,0.978167
4,(ROSES REGENCY TEACUP AND SAUCER),(GREEN REGENCY TEACUP AND SAUCER),0.058314,0.057197,0.042882,0.735360,12.856564,0.039546,3.562591,0.979327
5,(PINK REGENCY TEACUP AND SAUCER),(ROSES REGENCY TEACUP AND SAUCER),0.045640,0.058314,0.035658,0.781295,13.398153,0.032997,4.305737,0.969616
6,(SMALL MARSHMALLOWS PINK BOWL),(SMALL DOLLY MIX DESIGN ORANGE BOWL),0.027646,0.032506,0.021605,0.781473,24.040941,0.020706,4.427337,0.985654
7,(WOODEN HEART CHRISTMAS SCANDINAVIAN),(WOODEN STAR CHRISTMAS SCANDINAVIAN),0.031587,0.029485,0.023181,0.733888,24.890072,0.022250,3.647013,0.991130
8,(WOODEN STAR CHRISTMAS SCANDINAVIAN),(WOODEN HEART CHRISTMAS SCANDINAVIAN),0.029485,0.031587,0.023181,0.786192,24.890072,0.022250,4.529350,0.988984
9,"(PINK REGENCY TEACUP AND SAUCER, GREEN REGENCY...",(ROSES REGENCY TEACUP AND SAUCER),0.037628,0.058314,0.032178,0.855148,14.664638,0.029983,6.501040,0.968242
